## requirements

In [3]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3 konlpy pandas tqdm
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

Reading package lists... Done
W: chmod 0700 of directory /var/lib/apt/lists/partial failed - SetupAPTPartialDirectory (1: Operation not permitted)
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
  Using cached JPype1-py3-0.5.5.4.tar.gz (88 kB)
     |████████████████████████████████| 67 kB 764 kB/s eta 0:00:011
  ERROR: Command errored out with exit status 1:
   command: /home/nine/venvs/proj1/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-qra6hi9_/JPype1-py3/setup.py'"'"'; __file__='"'"'/tmp/pip-in

Failed to build JPype1-py3
    Running setup.py install for JPype1-py3 ... error
    ERROR: Command errored out with exit status 1:
     command: /home/nine/venvs/proj1/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-qra6hi9_/JPype1-py3/setup.py'"'"'; __file__='"'"'/tmp/pip-install-qra6hi9_/JPype1-py3/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-4dvgnszi/install-record.txt --single-version-externally-managed --compile --install-headers /home/nine/venvs/proj1/include/site/python3.5/JPype1-py3
         cwd: /tmp/pip-install-qra6hi9_/JPype1-py3/
    Complete output (64 lines):
    
    ********* DEPRECATION WARNING *********
    ********* DEPRECATION WARNING *********
    
    running install
    running build
    running build_py
    creating build
    creating build/lib.linux-x86_64-3.5


## read data

In [1]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/fllff/nlp_basic_lec/master/ratings_train.txt',sep='\t')

# NaN 삭제
df = df.dropna()

print(df)

              id                                           document  label
0        9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1        3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2       10265843                                  너무재밓었다그래서보는것을추천한다      0
3        9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4        6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
5        5403919      막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.      0
6        7797314                              원작의 긴장감을 제대로 살려내지못했다.      0
7        9443947  별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...      0
8        7156791                             액션이 없는데도 재미 있는 몇안되는 영화      1
9        5912145      왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?      1
10       9008700                                   걍인피니트가짱이다.진짜짱이다♥      1
11      10217543        볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~      1
12       5957425         

## tokenization

In [2]:
from konlpy.tag import Mecab
mecab = Mecab()

mecab.morphs('한국어 토큰 분리기 사용방법')


ImportError: /home/nine/venvs/proj1/lib/python3.5/site-packages/_jpype.cpython-35m-x86_64-linux-gnu.so: undefined symbol: PySlice_Unpack

In [ ]:
from tqdm import tqdm

for i, row in tqdm(df.iterrows()):
    df.at[i,'document'] = mecab.morphs(row['document'])
    
print(df)                                

In [ ]:
doc_list = df['document']
label_list = df['label']

doc_len_list = [len(doc) for doc in doc_list]

# 문장길이 분포 확인
distribution = [0]*20
for doc_len in doc_len_list:
    distribution[int(doc_len/10)] += 1
print(distribution)

# label균형 확인
label_list.value_counts()

In [ ]:
# 문장 길이 trim
doc_list = [doc[:50] for doc in doc_list]

# 문장길이 분포 확인
distribution = [0]*20
for doc in doc_list:
    distribution[int(len(doc)/10)] += 1
print(distribution)

## vectorization


In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(doc_list, size=16, window=5, min_count=50, workers=2, iter=3)


In [ ]:
model.wv.most_similar('배우')

In [ ]:
model.wv['배우']

In [ ]:
# token -> vector 변환
vector_list = []
for doc in tqdm(doc_list):
    tmp = []
    for token in doc:
        try:
            tmp.append(model.wv[token].tolist())
        except:
            tmp.append([0]*10)
        
    vector_list.append(tmp)
    
print(vector_list[0])


In [ ]:
MAX_LEN = 50
tmp=[]

for i, vector in tqdm(enumerate(vector_list)):
    vector_list[i].extend([[0]*16 for x in range(MAX_LEN-len(vector))])

print(len(vector_list[0]))

## Model

In [1]:
import tensorflow as tf
tf.__version__

'2.2.0'